<a href="https://colab.research.google.com/github/r-u-5k/study-dev/blob/master/5-1-type2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 신용카드서비스를 떠나는 고객을 찾아라
- 나이, 급여, 결혼 상태, 신용 카드 한도, 신용 카드 카테고리 등의 컬럼이 있음
- 평가: ROC-AUC, 정확도(Accuracy), F1, 정밀도(Precision), 재현율(Recall)을 구하시오
- target : Attrition_Flag (1:이탈, 0:유지)
- csv파일 생성 : 수험번호.csv (예시 아래 참조)
~~~
CLIENTNUM,Attrition_Flag
788544108,0.633
719356008,0.123
712142733,0.355
~~~


### 평가(분류)
~~~
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

# 정확도
print(accuracy_score(y_test, pred))

# 정밀도
print(precision_score(y_test, pred))

# 재현율 (민감도)
print(recall_score(y_test, pred))

# F1
print(f1_score(y_test , pred))

# roc-auc
print(roc_auc_score(y_test, pred_proba))
~~~

## 데이터 불러오기

In [ ]:
import pandas as pd

train = pd.read_csv("https://raw.githubusercontent.com/lovedlim/inf/main/p2/ceredit%20card/train.csv")
test = pd.read_csv("https://raw.githubusercontent.com/lovedlim/inf/main/p2/ceredit%20card/test.csv")
# train = pd.read_csv("train.csv")
# test =  pd.read_csv("test.csv")


In [ ]:
train.head()

,CLIENTNUM,Customer_Age,Gender,Dependent_count,Education_Level,Marital_Status,Income_Category,Card_Category,Months_on_book,Total_Relationship_Count,...,Contacts_Count_12_mon,Credit_Limit,Total_Revolving_Bal,Avg_Open_To_Buy,Total_Amt_Chng_Q4_Q1,Total_Trans_Amt,Total_Trans_Ct,Total_Ct_Chng_Q4_Q1,Avg_Utilization_Ratio,Attrition_Flag
0,778461408,38,M,1,High School,Married,$80K - $120K,Blue,23,4,...,2,22149.0,987,21162.0,1.633,2409,36,0.800,0.045,0
1,718263108,44,F,3,Unknown,Divorced,Less than $40K,Blue,32,3,...,2,1693.0,1457,236.0,0.919,2829,42,0.615,0.861,1
2,780384483,49,M,4,Unknown,Unknown,$80K - $120K,Blue,39,2,...,2,2886.0,925,1961.0,0.946,5481,58,0.758,0.321,1
3,719592483,43,F,5,Graduate,Single,Unknown,Blue,34,6,...,3,4607.0,2440,2167.0,0.646,4825,77,0.878,0.530,0
4,771453033,47,M,3,High School,Married,$60K - $80K,Blue,35,5,...,2,17343.0,1280,16063.0,1.314,3170,57,0.629,0.074,0


In [ ]:
train.describe(include='O')

,Gender,Education_Level,Marital_Status,Income_Category,Card_Category
count,8101,8101,8101,8101,8101
unique,2,7,4,6,4
top,F,Graduate,Married,Less than $40K,Blue
freq,4304,2481,3775,2860,7543


In [ ]:
train.shape, test.shape

((8101, 21), (2026, 20))

In [ ]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8101 entries, 0 to 8100
Data columns (total 21 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   CLIENTNUM                 8101 non-null   int64  
 1   Customer_Age              8101 non-null   int64  
 2   Gender                    8101 non-null   object 
 3   Dependent_count           8101 non-null   int64  
 4   Education_Level           8101 non-null   object 
 5   Marital_Status            8101 non-null   object 
 6   Income_Category           8101 non-null   object 
 7   Card_Category             8101 non-null   object 
 8   Months_on_book            8101 non-null   int64  
 9   Total_Relationship_Count  8101 non-null   int64  
 10  Months_Inactive_12_mon    8101 non-null   int64  
 11  Contacts_Count_12_mon     8101 non-null   int64  
 12  Credit_Limit              8101 non-null   float64
 13  Total_Revolving_Bal       8101 non-null   int64  
 14  Avg_Open

In [ ]:
train['Attrition_Flag'].value_counts()

,count
Attrition_Flag,
0,6815
1,1286


## 데이터 전처리 & 피처엔지니어링

In [ ]:
train.shape, test.shape

((8101, 21), (2026, 20))

In [ ]:
cols = train.select_dtypes(include='object').columns
train = train.drop(cols, axis=1)
test = test.drop(cols, axis=1)

# print(train.shape, test.shape)
# train = train.drop(cols, axis=1)
# test = test.drop(cols, axis=1)
# print(train.shape, test.shape)
# # train.drop(cols, axis=1, inplace=True)

In [ ]:
train.shape, test.shape

((8101, 16), (2026, 15))

In [ ]:
target = train.pop('Attrition_Flag')

## 검증 데이터 분리

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(train, target, test_size=0.2, random_state=0)

In [ ]:
X_train.shape, X_val.shape, y_train.shape, y_val.shape

((6480, 15), (1621, 15), (6480,), (1621,))

## 모델 & 평가

In [ ]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(random_state=0)
model.fit(X_train, y_train)
pred = model.predict(X_val)
pred_proba = model.predict_proba(X_val)

In [ ]:
pred[:10]

array([0, 0, 0, 0, 0, 1, 0, 0, 0, 0])

In [ ]:
pred_proba[:10]

array([[0.99, 0.01],
       [0.98, 0.02],
       [0.92, 0.08],
       [0.99, 0.01],
       [1.  , 0.  ],
       [0.16, 0.84],
       [0.94, 0.06],
       [0.86, 0.14],
       [0.97, 0.03],
       [1.  , 0.  ]])

In [ ]:
print(model.classes_)

[0 1]


In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

# 정확도
print(accuracy_score(y_val, pred))

# 정밀도
print(precision_score(y_val, pred))

# 재현율 (민감도)
print(recall_score(y_val, pred))

# F1
print(f1_score(y_val, pred))

# roc-auc
print(roc_auc_score(y_val, pred_proba[:, 1]))

0.9666872301048736
0.9488372093023256
0.8259109311740891
0.8831168831168831
0.989091809133179


## 예측 및 csv 제출

In [ ]:
pred = model.predict(test)
pred_proba = model.predict_proba(test)

In [ ]:
pred[:10]

array([0, 0, 0, 0, 0, 1, 1, 0, 1, 1])

In [ ]:
pred_proba[:10]

array([[1.  , 0.  ],
       [0.97, 0.03],
       [0.89, 0.11],
       [0.99, 0.01],
       [1.  , 0.  ],
       [0.03, 0.97],
       [0.19, 0.81],
       [1.  , 0.  ],
       [0.49, 0.51],
       [0.08, 0.92]])

In [ ]:
submit = pd.DataFrame({'CLIENTNUM': test['CLIENTNUM'], 'Attrition_Flag': pred_proba[:, 1]})
submit

,CLIENTNUM,Attrition_Flag
0,788544108,0.00
1,719356008,0.03
2,712142733,0.11
3,789394833,0.01
4,742778358,0.00
...,...,...
2021,787361583,0.08
2022,713474733,0.77
2023,708486033,0.33
2024,717683583,0.02


In [ ]:
submit.to_csv('result.csv', index=False)

In [ ]:
pd.read_csv('result.csv')

,CLIENTNUM,Attrition_Flag
0,788544108,0.00
1,719356008,0.03
2,712142733,0.11
3,789394833,0.01
4,742778358,0.00
...,...,...
2021,787361583,0.08
2022,713474733,0.77
2023,708486033,0.33
2024,717683583,0.02


##정리

In [ ]:
# 아래 코드는 실제 시험에서는 확인할 수 없는 부분입니다.
# y_test = pd.read_csv("https://raw.githubusercontent.com/lovedlim/inf/main/p2/ceredit%20card/y_test.csv")
# print(roc_auc_score(y_test, pred[:,1]))